<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#New-Hill-climbing-Data" data-toc-modified-id="New-Hill-climbing-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>New Hill-climbing Data</a></span></li></ul></div>

In [9]:
import random
import matplotlib.pyplot as plt
from scipy.stats import norm, stats
import numpy as np
from statistics import mean

In [10]:
# Same as original
def generate_obs(x):
    "Return a single point from the distributino"
    return (0.3 * norm(35, 5).pdf(x)  + \
            0.7 * norm(60, 5).pdf(x)) * \
                200

# Same as original
def query_data(lower=20, upper=80, n_obs=100, verbose=False, seed=1):
    " Generate a list of results of the generate_obs() function from random inputs"
    
    random.seed(seed)
    
    # Use random.uniform() to create random values and create a list
    observations = []
    x = [random.uniform(lower, upper) for _ in range(n_obs)]
    
    # Sort the list.  That will make our plot actually readable.
    #   NB:  x.sort() returns None so we can't add sort() to our definition of x
    x.sort()
    
    # Runs through our random inputs and remenbers the results
    for xval in x:                
        obs = generate_obs(xval)
        observations.append(obs) 
        if verbose:  
            print(f"f({xval:2.3f}) = {obs:2.3f}")
            
    return x, observations  

# Same as original
def get_max_xy(xy_arr):
    " Input is two-col ndarray.  Cols: [x, f(x)].  Returns x and f(x) at max f(x) value."
    
    max_row = xy_arr[np.where(xy[:,1] == np.max(xy_arr[:,1]))][0]
    x_max, y_max = max_row[0], max_row[1]

    return x_max, y_max

# Same as original
def next_x_vals(guess, frame=3, step=1, direction='up', samples=None):
    """Create and return a vector such that each element is 'step' different
         from the last, starting at guess +/- step, depending on the 
         'direction'.  The length of the vector is 'frame'.  """
    
    return_list = []
    for ix in range(frame ):
        if direction == 'up':
            return_list.append(guess + ix * step)
        else:
            return_list.append(guess - ix * step)
            
    return return_list
            
 # Same as original   
def next_y_vals(next_x_vals, func):
    """ Given a vector of next_x_vals and a function, create and return a vector of f(x)."""
    
    return_vector = []
    
    for x in next_x_vals:
        return_vector.append(func(x))        
    return return_vector

def ORIGINAL_gradient_search_max(guess, func, lower_bound, upper_bound, step, frame, direction=None):
    """Conduct a gradient search to find max value of a solution space."""  
    
    DEBUG = False
    
    # Remember the oringinal guess
    original_guess = guess
    original_result = func(original_guess)
    if DEBUG: print(f"current: x {original_guess:.2f}   y {original_result:.2f}")
    
    # Find x values for a few steps higher and lower than current guess.
    x_going_up = next_x_vals(guess, frame, step, 'up')
    x_going_down = next_x_vals(guess, frame, step, 'down')
    if DEBUG: print(x_going_up, x_going_down)
    
    
    # Find corresponding mean y values
    y_going_up = mean(next_y_vals(x_going_up, func))
    y_going_down = mean(next_y_vals(x_going_down, func))
    if DEBUG: print(f"y_going_down: {y_going_down:.2f}  y_going_up: {y_going_up:.2f} ")
    
    
    # Figure out if we're going up or down
    if direction is None:
        if y_going_up >= original_result and y_going_up >= y_going_down:
            direction = 'up'
        if y_going_down > original_result and y_going_up < y_going_down:
            direction = 'down' 
        print(f"We're searching {direction}.")
    
    # Adjust the guess as needed
    if direction == 'up' and y_going_up >= original_result:
        guess = original_guess + step
    elif direction == 'down'  and y_going_down >= original_result:
        guess = original_guess - step
            
    # We'll keep going if the guess changed and it's within bounds. Otherwise we're done.
    if guess != original_guess and guess >= lower_bound and guess <= upper_bound:
        print(f"Current answer:  x: {guess:.2f}  y: {func(guess):.2f}")
        gradient_search_max(guess, func, lower_bound, upper_bound, step, frame, direction=direction)
        
    else:
        print(f"Final answer:  x: {guess:.2f}  y: {func(guess):.2f}")
        return guess

In [11]:
# Create our actual PDF (probability density function)
X_plot = np.linspace(10, 100, 1000)[:, np.newaxis]

true_dens = (0.3 * norm(35, 5).pdf(X_plot[:, 0])  + \
             0.7 * norm(60, 5).pdf(X_plot[:, 0])) * \
            200

In [12]:
# Set our search range
n_obs = 10    
lower = 20
upper = 75

# Query the data
x, obs = query_data (lower=lower, upper=upper, n_obs=n_obs, verbose=False) 
obs_array = np.array(obs)

# Get an array of the x,y values
xy = np.stack((np.array(x), obs_array), axis=1)

# Figure out the values of x and y where y is at its max
x_max, y_max = get_max_xy(xy)

# Set some initial values
guess0 = x_max
result0 = y_max
step = 1
frame = 3
guess = x_max

In [15]:
ORIGINAL_gradient_search_max(guess, func, lower_bound_gradient, upper_bound_gradient, step, frame)

We're searching down.
Current answer:  x: 61.01  y: 10.95
Current answer:  x: 60.01  y: 11.17
Final answer:  x: 60.01  y: 11.17


In [16]:
# The actual max of the true PDF:
print(f"true max value: {stats.describe(true_dens).minmax[1]:.2f}")

true max value: 11.17
